In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [20]:
tag = 'vacation'
browser = webdriver.Chrome('/Users/matt/Downloads/chromedriver')
browser.get('https://twitter.com/hashtag/' + tag + '?f=tweets&vertical=default&lang=en')

In [21]:
links=[]

def scrape(start, end):
    print('scrolling: {}-{}'.format(start, end))
    Pagelength = browser.execute_script("window.scrollTo({}, {});".format(start, end))
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.findAll('div', {'data-permalink-path': True})
    for div in script:
        status = div.get('data-permalink-path')
        links.append('https://www.twitter.com{}'.format(status))
    
    time.sleep(5)
    if (end < 700000):
        scrape(start + 5000, end + 5000)

In [22]:
scrape(0, 5000)

scrolling: 0-5000
scrolling: 5000-10000
scrolling: 10000-15000
scrolling: 15000-20000
scrolling: 20000-25000
scrolling: 25000-30000
scrolling: 30000-35000
scrolling: 35000-40000
scrolling: 40000-45000
scrolling: 45000-50000
scrolling: 50000-55000
scrolling: 55000-60000
scrolling: 60000-65000
scrolling: 65000-70000
scrolling: 70000-75000
scrolling: 75000-80000
scrolling: 80000-85000
scrolling: 85000-90000
scrolling: 90000-95000
scrolling: 95000-100000
scrolling: 100000-105000
scrolling: 105000-110000
scrolling: 110000-115000
scrolling: 115000-120000
scrolling: 120000-125000
scrolling: 125000-130000
scrolling: 130000-135000
scrolling: 135000-140000
scrolling: 140000-145000
scrolling: 145000-150000
scrolling: 150000-155000
scrolling: 155000-160000
scrolling: 160000-165000
scrolling: 165000-170000
scrolling: 170000-175000
scrolling: 175000-180000
scrolling: 180000-185000
scrolling: 185000-190000
scrolling: 190000-195000
scrolling: 195000-200000
scrolling: 200000-205000
scrolling: 205000-21

In [23]:
unique = list(set(links))
print('Found {} tweets'.format(len(unique)))

Found 658 tweets


In [24]:
result = pd.DataFrame()
cols = ['user', 'location', 'geoLocation', 'tweet', 'timeStamp', 'replies', 'retweets', 'favorites', 'url']

for i in range(len(unique)):
    try:
        print(i, end=' ')
        page = urlopen(unique[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        user = body.find('b', {'class': 'u-linkComplex-target'})
        location = body.find('span', {'class': 'ProfileHeaderCard-locationText u-dir'})
        content = body.find('p', {'class': 'tweet-text'})
        tweetDetails = body.find('div', {'class': 'tweet-details-fixer'})
        timeStamp = tweetDetails.find('span').find('span').text
        
        geoLocation = body.find('span', {'class': 'permalink-tweet-geo-text'})
        gl = '' if geoLocation == None else geoLocation.find('a').text
        
        replyContainer = body.find('div', {'class': 'ProfileTweet-action--reply'})
        replies = replyContainer.find('span', {'class': 'ProfileTweet-actionCount'}).find('span').text
        
        retweetContainer = body.find('div', {'class': 'ProfileTweet-action--retweet'})
        retweets = retweetContainer.find('span', {'class': 'ProfileTweet-actionCount'}).find('span').text
        
        favoriteContainer = body.find('div', {'class': 'ProfileTweet-action--favorite'})
        favorites = favoriteContainer.find('span', {'class': 'ProfileTweet-actionCount'}).find('span').text
        
        tweet = {
            'user': '@{}'.format(user.text),
            'location': location.text.strip(),
            'geoLocation': gl,
            'tweet': content.text,
            'timeStamp': timeStamp,
            'replies': replies or 0,
            'retweets': retweets or 0,
            'favorites': favorites or 0,
            'url': unique[i]
        }
        x = pd.DataFrame.from_dict(json_normalize(tweet), orient='columns') 
        x = x[cols]
        result = result.append(x) 
    except:
        np.nan

result.index = range(len(result.index))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [25]:
result = result.drop_duplicates()
print('scraped {} tweets'.format(len(result)))
result.head()

scraped 658 tweets


,user,location,geoLocation,tweet,timeStamp,replies,retweets,favorites,url
0,@valencia_travel,Cusco - Peru,,“If somebody offers you an amazing opportunity...,8:27 AM - 29 Nov 2019,0,0,0,https://www.twitter.com/valencia_travel/status...
1,@bestforluxury,"London, England",,#Travel #Flights #Hotel #Holidays #Getaway #Va...,1:43 AM - 29 Nov 2019,0,0,0,https://www.twitter.com/bestforluxury/status/1...
2,@bestravel3,Worldwide!,,Convenient Sports Stereo Wired Earphones http:...,7:46 AM - 29 Nov 2019,0,0,0,https://www.twitter.com/bestravel3/status/1200...
3,@blogosum,,,Travel #news: Malaysia Airlines expands UK tea...,1:31 AM - 29 Nov 2019,0,0,0,https://www.twitter.com/blogosum/status/120034...
4,@Logalvezantoine,,,Siempre habrán lugares inolvidables...\nRecuer...,3:49 AM - 29 Nov 2019,2,0,20,https://www.twitter.com/Logalvezantoine/status...


In [26]:
result.to_csv('vacation-output.csv', columns=cols, index=None)